Apriori Algorithm

In [401]:
import os
while True:
    try:
        support = float(input('Enter support in percentage:'))
        if support < 1 or support > 100:
            raise ValueError 
        break
    except ValueError:
        print("Invalid support value.The number must be in the range of 1-100.")

while True:
    try:
        confidence = float(input('Enter confidence in percentage:'))
        if confidence < 1 or confidence > 100:
            raise ValueError 
        break
    except ValueError:
        print("Invalid confidence. The number must be in the range of 1-100.")

min_support = support/100
min_confidence=confidence/100

Enter support in percentage:40
Enter confidence in percentage:100


Getting Data From text File

In [402]:
while True:
    try:
        s=int(input("'Enter number from below to choose database: 1.Amazon, 2.Nike, 3.BestBuy, 4.dataset1,5.KMart"))
        if s < 1 or s > 5:
            raise ValueError 
        break
    except ValueError:
        print("Invalid support value.The number must be in the range of 1-5.")

dataset_dict= {1:'Amazon.txt',2:'Nike.txt',3:'BestBuy.txt',4:'dataset1.txt',5:'KMart.txt'}
path_to_data = dataset_dict[s]
print(path_to_data)

'Enter number from below to choose database: 1.Amazon, 2.Nike, 3.BestBuy, 4.dataset1,5.KMart5
KMart.txt


In [403]:
def load_transactions(path_to_data):
    Transactions = []
    order=[]
    with open(path_to_data,'r') as fid:
        for lines in fid:
            str_line = list(lines.strip().split(','))
            
            unq_line = list(set(str_line))
            unq_line.sort()
            for each_ele in unq_line:
                order.append(each_ele)
            Transactions.append(unq_line)
        order=list(set(order))
        order.sort()
    return Transactions,order
            

In [404]:
pd=load_transactions(path_to_data)
Transactions=pd[0]
order=pd[1]
num_trans = len(Transactions)
print('order\n\n',order)
print('\nTransactions\n')
Transactions

order

 ['Bed Skirts', 'Bedding Collections', 'Bedspreads', 'Decorative Pillows', 'Embroidered Bedspread', 'Kids Bedding', 'Quilts', 'Shams', 'Sheets']

Transactions



[['Decorative Pillows', 'Embroidered Bedspread', 'Quilts'],
 ['Bed Skirts',
  'Bedding Collections',
  'Bedspreads',
  'Embroidered Bedspread',
  'Kids Bedding',
  'Shams',
  'Sheets'],
 ['Bedding Collections',
  'Decorative Pillows',
  'Embroidered Bedspread',
  'Kids Bedding',
  'Quilts',
  'Shams'],
 ['Bed Skirts', 'Bedding Collections', 'Bedspreads', 'Kids Bedding', 'Sheets'],
 ['Bed Skirts',
  'Bedding Collections',
  'Bedspreads',
  'Decorative Pillows',
  'Kids Bedding',
  'Sheets'],
 ['Bed Skirts',
  'Bedding Collections',
  'Bedspreads',
  'Kids Bedding',
  'Shams',
  'Sheets'],
 ['Decorative Pillows', 'Quilts'],
 ['Decorative Pillows', 'Embroidered Bedspread', 'Quilts'],
 ['Bed Skirts', 'Bedspreads', 'Kids Bedding', 'Shams', 'Sheets'],
 ['Bedding Collections', 'Embroidered Bedspread', 'Quilts'],
 ['Bed Skirts',
  'Bedding Collections',
  'Bedspreads',
  'Kids Bedding',
  'Shams',
  'Sheets'],
 ['Decorative Pillows', 'Quilts'],
 ['Embroidered Bedspread', 'Shams'],
 ['Bed Skirt

Generating itemset-1 

In [405]:
C={}
L={}
itemset_size=1

Discarded = {itemset_size:[]}
C.update({itemset_size : [[f] for f in order]})

    


In [406]:
def count_occurences(itemset,Transactions):
    count=0
    for i in range(len(Transactions)):
        if set(itemset).issubset(set(Transactions[i])):
            count+=1
            
    return count         

In [407]:
def get_frequent(itemsets,Tansactions,min_support,prev_discarded):
    L=[]
    supp_count=[]
    new_discarded=[]
    num_trans= len(Transactions)
    
    k= len(prev_discarded.keys())
    
    for s in range(len(itemsets)):
        discarded_before = False
        if k>0:
            for it in prev_discarded[k]:
                if set(it).issubset(set(itemsets[s])):
                    discarded_before=True
                    break
                    
        if not discarded_before:
            count=count_occurences(itemsets[s],Transactions)
            if count/num_trans >= min_support:
                L.append(itemsets[s])
                supp_count.append(count)
            else:
                new_discarded.append(itemsets[s])
                
    return L,supp_count,new_discarded
    

In [408]:
supp_count_L={}
f,sup,new_discarded=get_frequent(C[itemset_size],Transactions,min_support,Discarded)
Discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size:sup})

function for Printing Table

In [409]:
def print_table(T,supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
        print("{} : {}".format(T[k],supp_count[k]))
    print("\n\n")    

In [410]:
print_table(L[1],supp_count_L[1])

Itemset | Frequency
['Bed Skirts'] : 11
['Decorative Pillows'] : 10
['Kids Bedding'] : 12
['Quilts'] : 8
['Shams'] : 11
['Sheets'] : 10





Generating itemset-k

In [411]:
def join_two_itemsets(it1,it2,order):
    it1.sort(key=lambda x:order.index(x))
    it2.sort(key=lambda x:order.index(x))
    
    for i in range(len(it1)-1):
        if it1[i] != it2[i]:
            return []
    
        
    if order.index(it1[-1]) < order.index(it2[-1]):
        return it1 + [it2[-1]]
    
    return []


In [412]:
def join_itemsets(set_of_its,order):
    c=[]
    for i in range(len(set_of_its)):
        for j in range(i+1,len(set_of_its)):
            it_out = join_two_itemsets(set_of_its[i],set_of_its[j],order)
            if len(it_out) >0:
                c.append(it_out)
                
    return c             
                                                  

In [413]:
k=itemset_size+1
convergence =False
while not convergence:
    C.update({k : join_itemsets(L[k-1],order)})
    
    print("Table c{}: \n".format(k))
    print_table(C[k],[count_occurences(it,Transactions) for it in C[k]])
    f,sup,new_discarded=get_frequent(C[k],Transactions,min_support,Discarded)
    Discarded.update({k:new_discarded})
    L.update({k:f})
    supp_count_L.update({k:sup})
    if len(L[k]) ==0:
        convergence=True
    else:
        print("Table L{}: \n".format(k))
        print_table(L[k],supp_count_L[k])
    k+=1
    

Table c2: 

Itemset | Frequency
['Bed Skirts', 'Decorative Pillows'] : 4
['Bed Skirts', 'Kids Bedding'] : 10
['Bed Skirts', 'Quilts'] : 0
['Bed Skirts', 'Shams'] : 9
['Bed Skirts', 'Sheets'] : 9
['Decorative Pillows', 'Kids Bedding'] : 4
['Decorative Pillows', 'Quilts'] : 6
['Decorative Pillows', 'Shams'] : 4
['Decorative Pillows', 'Sheets'] : 2
['Kids Bedding', 'Quilts'] : 2
['Kids Bedding', 'Shams'] : 9
['Kids Bedding', 'Sheets'] : 10
['Quilts', 'Shams'] : 1
['Quilts', 'Sheets'] : 1
['Shams', 'Sheets'] : 7



Table L2: 

Itemset | Frequency
['Bed Skirts', 'Kids Bedding'] : 10
['Bed Skirts', 'Shams'] : 9
['Bed Skirts', 'Sheets'] : 9
['Kids Bedding', 'Shams'] : 9
['Kids Bedding', 'Sheets'] : 10



Table c3: 

Itemset | Frequency
['Bed Skirts', 'Kids Bedding', 'Shams'] : 8
['Bed Skirts', 'Kids Bedding', 'Sheets'] : 9
['Bed Skirts', 'Shams', 'Sheets'] : 7
['Kids Bedding', 'Shams', 'Sheets'] : 7



Table L3: 

Itemset | Frequency
['Bed Skirts', 'Kids Bedding', 'Shams'] : 8
['Bed Skirts', 

Generating Association Rules

In [414]:
from itertools import combinations,chain
def powerset(s):
    return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1)))
    

In [415]:
def write_rules(X,X_S,S,conf,supp,num_tran):
    out_rules = ""
    out_rules += "Frequent Itemset: {}\n".format(X)
    out_rules +="     Rule: {} -> {} \n".format(list(S),list(X_S)) 
    out_rules +="     Confidence: {0:2.3f}".format(conf)
    out_rules +="     Support: {0:2.3f}\n".format(supp/num_trans)
    return out_rules
    

In [416]:
assoc_rules_str = ""
for i in range(1,len(L)):
    for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        
        for z in s:
            S = set(z)
            X = set(L[i][j])
            X_S = set(X-S)
            sup_x = count_occurences(X,Transactions)
            sup_x_s = count_occurences(X_S,Transactions)
            conf = sup_x / count_occurences(S,Transactions)
            if conf >= min_confidence and sup_x >= min_support:
                assoc_rules_str += write_rules(X,X_S,S,conf,sup_x,num_trans)
        

In [417]:
print(assoc_rules_str)

Frequent Itemset: {'Kids Bedding', 'Sheets'}
     Rule: ['Sheets'] -> ['Kids Bedding'] 
     Confidence: 1.000     Support: 0.500
Frequent Itemset: {'Kids Bedding', 'Bed Skirts', 'Sheets'}
     Rule: ['Bed Skirts', 'Sheets'] -> ['Kids Bedding'] 
     Confidence: 1.000     Support: 0.450

